In [ ]:
# all the libraries and dependencies imported initially
import cv2
import numpy as np
import sqlite3
import bcrypt
import time
from keras_facenet import FaceNet
from mtcnn import MTCNN

detection_method = MTCNN()
embed_extract = FaceNet()

# Database connection with exception handling in case the 'database is locked' error apprears
def connect_db_with_exp_handling(retries=5, delay=1):
    for i in range(retries):
        try:
            conn = sqlite3.connect('user_face_data.db')  # Changed database name
            return conn
        except sqlite3.OperationalError as e:
            if "database is locked" in str(e):
                print(f"Database is locked, retrying... ({i+1}/{retries})")
                time.sleep(delay)
            else:
                raise e
    raise sqlite3.OperationalError("Database is still locked after retries.")

# Clear the existing tables to avoid any conflits,creating new tables if they don't already exists
def initial_db_conn():
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        
        c.execute("DROP TABLE IF EXISTS user_info")  
        c.execute("DROP TABLE IF EXISTS face_embeddings")  
        c.execute("DROP TABLE IF EXISTS payment_methods") 
        
        # Create new tables for storing user data, face embeddings, and payment methods
        # Users table for storing name, email, and hashed password
        c.execute('''CREATE TABLE IF NOT EXISTS user_info 
                     (id INTEGER PRIMARY KEY, name TEXT, email TEXT, password TEXT)''')
        # Faces table for storing face embeddings
        c.execute('''CREATE TABLE IF NOT EXISTS face_embeddings 
                     (user_id INTEGER, encoding BLOB, 
                      FOREIGN KEY(user_id) REFERENCES user_info(id))''')
        # Payment methods table for storing card/bank details
        c.execute('''CREATE TABLE IF NOT EXISTS payment_methods 
                     (id INTEGER PRIMARY KEY, user_id INTEGER, card_number TEXT, bank_name TEXT, 
                      card_type TEXT, expiry_date TEXT, 
                      FOREIGN KEY(user_id) REFERENCES user_info(id))''')
        conn.commit()

# Capture image from webcam
def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret, frame = video_cap.read()
    video_cap.release()
    return frame

# Detect face from the image
def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

# Extract face region from the image
def ext_face(img, box):
    x, y, wt, ht = box
    face = img[y:y+ht, x:x+wt]
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32') / 255
    return face

# Extract face embedding
def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

# Register user and store user details, face embedding along with payment methods
def register_user(name, email, password, embedding, payment_methods):
    try:
        with connect_db_with_exp_handling() as conn:
            c = conn.cursor()
            hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()) 
            # Insert user information into users table
            c.execute("INSERT INTO user_info (name, email, password) VALUES (?, ?, ?)", (name, email, hashed_password))
            user_id = c.lastrowid  # Get the inserted user's ID
            # Insert face embedding into face_embeddings table
            c.execute("INSERT INTO face_embeddings (user_id, encoding) VALUES (?, ?)", (user_id, embedding.tobytes()))
            # Insert payment methods of users into payment_methods table
            for payment_method in payment_methods:
                c.execute('''INSERT INTO payment_methods (user_id, card_number, bank_name, card_type, expiry_date) 
                             VALUES (?, ?, ?, ?, ?)''', 
                          (user_id, payment_method['card_number'], payment_method['bank_name'], 
                           payment_method['card_type'], payment_method['expiry_date']))
            conn.commit()
    except sqlite3.OperationalError as e:
        print(f"Database error: {e}")

# load users info by their stored details during registration
def load_user_by_email(email):
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        c.execute("SELECT id, name, email, password FROM user_info WHERE email = ?", (email,))
        user = c.fetchone()
    return user


#load users face embedding based on the user id
def load_face_by_user_id(user_id):
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        c.execute("SELECT encoding FROM face_embeddings WHERE user_id = ?", (user_id,))
        rows = c.fetchall()
    face_embeddings = [np.frombuffer(row[0], dtype=np.float32) for row in rows]
    return face_embeddings

# load payment method based on the registation data of payment
def load_payment_methods_by_user_id(user_id):
    with connect_db_with_exp_handling() as conn:
        c = conn.cursor()
        c.execute("SELECT id, card_number, bank_name, card_type, expiry_date FROM payment_methods WHERE user_id = ?", (user_id,))
        rows = c.fetchall()
    payment_methods = [{'id': row[0], 'card_number': row[1], 'bank_name': row[2], 'card_type': row[3], 'expiry_date': row[4]} for row in rows]
    return payment_methods

# Calculate distance between two embeddings for the real-time verification
def calc_dist(embed1, embed2):
    return np.linalg.norm(embed1 - embed2)

# Registration process with face capture and payment methods input
def register():
    print("Please provide your details to sign up:")
    name = input("Name: ")
    email = input("Email: ")
    password = input("Password: ")

    # Capture face
    print("Now, capture your face:")
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = detect_img(rgb_image)

    if len(faces) == 0:
        print("No face detected. Please try again.")
        return

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)
    embedding = face_embed(face_image)

    # Collect payment methods from the user during registration
    payment_methods = []
    while True:
        print("\nAdd a payment method:")
        card_number = input("Enter card number: ")
        bank_name = input("Enter bank name: ")
        card_type = input("Enter card type (credit/debit): ")
        expiry_date = input("Enter expiry date (MM/YY): ")
        payment_methods.append({
            'card_number': card_number,
            'bank_name': bank_name,
            'card_type': card_type,
            'expiry_date': expiry_date
        })
        add_another = input("Do you want to add another payment method? (yes/no): ").lower()
        if add_another != 'yes':
            break

    # Store user information, face embedding, and payment methods
    register_user(name, email, password, embedding, payment_methods)
    print("Registration successful.")

# Login and  verify face for transaction
def login():
    print("Please login with your email and password:")
    email = input("Email: ")
    password = input("Password: ")

    user = load_user_by_email(email)
    if user is None:
        print("User not found. Please register.")
        return

    user_id, name, email, stored_password = user

    if bcrypt.checkpw(password.encode('utf-8'), stored_password):
        print(f"Welcome {name}!")

        # Asking if the user wants to proceed with a transaction
        transaction = input("Do you want to proceed with a transaction? (yes/no): ").lower()
        if transaction == "yes":
            # Load user's payment methods and select one for the transaction
            payment_methods = load_payment_methods_by_user_id(user_id)
            if len(payment_methods) == 0:
                print("No payment methods found. Please add a payment method.")
                return

            print("Select a payment method:")
            for idx, method in enumerate(payment_methods):
                print(f"{idx + 1}. {method['card_type']} Card - {method['card_number']} (Bank: {method['bank_name']}, Expiry: {method['expiry_date']})")
            
            selected_method_idx = int(input("Enter the number of the payment method to use: ")) - 1
            selected_method = payment_methods[selected_method_idx]

            # Capture and verify face for transaction
            verify_face(user_id)
            print(f"Payment with {selected_method['card_type']} Card ending in {selected_method['card_number'][-4:]} successful!")
        else:
            print("Login successful without face verification.")
    else:
        print("Incorrect password.")

# Face verification process
def verify_face(user_id):
    print("Please verify your face:")
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = detect_img(rgb_image)

    if len(faces) == 0:
        print("No face detected. Please try again.")
        return

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)
    embedding = face_embed(face_image)

    # Load stored embeddings for this user
    stored_embeddings = load_face_by_user_id(user_id)

    threshold = 0.5
    for stored_embedding in stored_embeddings:
        dist = calc_dist(embedding, stored_embedding)
        if dist < threshold:
            print(f"Face verified successfully. Distance: {dist:.4f}")
            return

    print("Face verification failed.")

# Main function 
def main():
    initial_db_conn()  # Initialize the database

    while True:
        print("1. Register")
        print("2. Login")
        choice = input("Enter your choice: ")

        if choice == '1':
            register()
        elif choice == '2':
            login()
        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


1. Register
2. Login
Enter your choice: 1
Please provide your details to sign up:
Name: Poulomi
Email: poulomisaha042@gmail.com
Password: Misti@123
Now, capture your face:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Add a payment method:
Enter card number: 000044445555566667777
Enter bank name: LLOYDS BANK
Enter card type (credit/debit): debit
Enter expiry date (MM/YY): 08/26
Do you want to add another payment method? (yes/no): no
Registration successful.
1. Register
2. Login
Enter your choice: 2
Please login with your email and password:
Emai